# Models

In [156]:
import numpy as np
from PuzzleLib.Backend import gpuarray
from PuzzleLib.Modules import Activation, sigmoid, relu, tanh, Linear, Reshape, BatchNorm1D
from PuzzleLib.Variable import Variable
from PuzzleLib.Containers import Graph, Sequential
import warnings


def generator_block(in_feat, out_feat):
    linear = Linear(in_feat, out_feat, name="fc").node()
    rs = Reshape((1, out_feat, -1)).node(linear)
    bn = BatchNorm1D(out_feat, 0.8).node(rs)
    act = Activation(relu, name="act").node(bn)
    rs2 = Reshape((-1, out_feat)).node(act)
    return Graph(inputs=linear, outputs=rs2)


def generator():
    layers = Sequential()
    layers.append(generator_block(32, 128))
    layers.append(generator_block(128, 256))
    layers.append(generator_block(256, 512))
    layers.append(generator_block(512, 1024))
    layers.append(Linear(1024, 32))
    layers.append(Activation(tanh))
    return layers

def discriminator():
    layers = Sequential()
    layers.append(Linear(32, 512))
    layers.append(Activation(relu))
    layers.append(Linear(512, 256))
    layers.append(Activation(relu))
    layers.append(Linear(256, 32))
    layers.append(Activation(sigmoid))
    return layers

In [157]:
D = discriminator()
G = generator()

# Setup criterion and optimizer

In [158]:
from PuzzleLib.Cost import BCE
from PuzzleLib.Optimizers import Adam

criterion = BCE()

optimizer_g = Adam()
optimizer_g.setupOn(G, useGlobalState=True)

optimizer_d = Adam()
optimizer_d.setupOn(D, useGlobalState=True)

# Dataset

In [159]:
from PuzzleLib.Datasets import Cifar10Loader

cifar10 = Cifar10Loader()
path = "./TestData/"
data, _ = cifar10.load(path=path)

# Train

In [160]:
for epoch in range(50):
    for i, img in enumerate(data):
        valid = gpuarray.to_gpu(np.ones((32, 32)).astype(np.int32))
        fake  = gpuarray.to_gpu(np.zeros((32, 32)).astype(np.int32))
        
        # -----------------
        #  Train Generator
        # -----------------
        optimizer_g.zeroGradParams()

        z = gpuarray.to_gpu(np.random.normal(0, 1, (32, 32)).astype(np.float32))
        gen_imgs = G(z)
        
        g_loss, grad = criterion(D(gen_imgs).ravel(), valid.ravel())
        G.backward(grad.reshape(32, 32))
        optimizer_g.update()
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        optimizer_d.zeroGradParams()
        
        real_loss, real_grad = criterion(D(gpuarray.to_gpu(img[0, ...])).ravel(), valid.ravel())
        fake_loss, fake_grad = criterion(D(gen_imgs).ravel(), fake.ravel())
        d_loss = real_loss + fake_loss
        d_grad = real_grad + fake_grad

        D.backward(d_grad.reshape(32, 32))
        optimizer_d.update()
        
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, 50, i, len(data), d_loss.item(), g_loss.item())
        )

[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 128), reshaped to (1, 128, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 128, 32), reshaped to (32, 128))
[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 256), reshaped to (1, 256, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 256, 32), reshaped to (32, 256))
[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 512), reshaped to (1, 512, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 512, 32), reshaped to (32, 512))
[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 1024), reshaped to (1, 1024, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 1024, 32), reshaped to (32, 1024))
[Epoch 0/50]

KeyboardInterrupt: 